In [7]:
import os, time, shutil
from pathlib import Path
from glob import glob
import numpy as np
import spikeinterface.full as si

In [2]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 5.5
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False
print(sorter_params)

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/phornauer/Git/Kilosort_2020b
{'detect_threshold': 5.5, 'projection_threshold': [10, 4], 'preclust_threshold': 8, 'car': True, 'minFR': 0.01, 'minfr_goodchannels': 0.01, 'nblocks': 5, 'sig': 20, 'freq_min': 150, 'sigmaMask': 30, 'nPCs': 3, 'ntbuff': 64, 'nfilt_factor': 4, 'NT': None, 'do_correction': False, 'wave_length': 61, 'keep_good_only': False, 'skip_kilosort_preprocessing': False, 'scaleproc': None, 'save_rez_to_mat': False, 'delete_tmp_files': True, 'delete_recording_dat': False, 'n_jobs': -1, 'chunk_duration': '1s', 'progress_bar': True}


In [3]:
input_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/'
path_list  = glob(input_path + '/M*/Network/*/data.raw.h5')

In [4]:
path_list 

['/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05588/Network/000102/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05737/Network/000098/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05579/Network/000096/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05866/Network/000100/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05865/Network/000094/data.raw.h5']

In [5]:
path_parts = path_list[0].split('/')
path_parts[0] = '/'
path_parts[6] = 'intermediate_data'
path_parts[7] = 'MaxTwo'
path_parts[8] = 'phornauer'
save_root = os.path.join(*path_parts[0:12])
save_root

'/net/bs-filesvr02/export/group/hierlemann/intermediate_data/MaxTwo/phornauer/iNeurons/230328/M05588'

In [ ]:
for p in path_list:
    print(p)
    path_parts = p.split('/')
    path_parts[0] = '/'
    path_parts[6] = 'intermediate_data'
    path_parts[8] = 'phornauer'
    save_root = os.path.join(*path_parts[0:12])
    for w in range(6):
        stream_name = 'well00' + str(w)
        save_path = os.path.join(save_root,stream_name)
        output_folder = Path(os.path.join(save_path, 'sorted'))
        
        if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
        #if not os.path.exists(output_folder):
            print(f"\n\n---------Sorting recording {os.path.join(path_parts[10],path_parts[11])} well {w+1}---------\n\n")
            output_folder.mkdir(parents=True, exist_ok=True)
            raw_file = os.path.join(output_folder,'recording.dat')
            wh_file = os.path.join(output_folder, 'temp_wh.dat')
            try:
                rec = si.MaxwellRecordingExtractor(p,stream_name=stream_name)
            except Exception:
                continue
            print(f"DURATION: {rec.get_num_frames() / rec.get_sampling_frequency()} s -- "
                  f"NUM. CHANNELS: {rec.get_num_channels()}")
            try:
                t_start_sort = time.time()
                sorting = si.run_sorter(sorter, rec, output_folder=output_folder, verbose=True,
                                        **sorter_params)
                print(f"\n\nSpike sorting elapsed time {time.time() - t_start_sort} s")
                if os.path.exists(wh_file):
                    os.remove(wh_file)
                if os.path.exists(raw_file):
                    os.remove(raw_file)
            except Exception:
                if os.path.exists(wh_file):
                    os.remove(wh_file)
                if os.path.exists(raw_file):
                    os.remove(raw_file)
                continue

/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230328/M05588/Network/000102/data.raw.h5


---------Sorting recording 230328/M05588 well 1---------


DURATION: 600.02 s -- NUM. CHANNELS: 1020


write_binary_recording:   0%|          | 0/601 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well000/sorted/sorter_output/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   8s. Loading raw data and applying filters... 

Time  78s. Finished preprocessing 92 batches. 

Drift correction DISABLED

pitch is 35 um

1.03 sec, 1 batches, 2121 spikes 

86.46 sec, 92 batches, 193944 s

In [8]:
# 17 
for p in path_list:
    path_parts = p.split('/')
    path_parts[0] = '/'
    path_parts[6] = 'intermediate_data'
    path_parts[8] = 'phornauer'
    save_root = os.path.join(*path_parts[0:12])
    failed_sortings = []
    for w in range(6):

        stream_name = 'well00' + str(w)
        save_path = os.path.join(save_root,stream_name)

        output_folder = Path(os.path.join(save_path, 'sorted'))
        if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
            print(output_folder)

/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well000/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well001/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well002/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well003/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well004/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05588/well005/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05737/well000/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230328/M05737/well001/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNe